# Buffer Overflow Protections
- modern kernel and compilers provide various proctections from bufferoverlow attacks
- Reference: https://ocw.cs.pub.ro/courses/cns/labs/lab-06

## 1. ASLR - Address Space Layout Randomization
- Kernel specific protection
- loads the stack, data and program in random memory locations making it difficult to guess their addresses

- the following segments are possible to randomize on x86:
    - **stack** is easily randomizable, as all stack addresses are relative to `esp` or `ebp`
    - **Data segment** may be randomized, if e.g., the data segment is set to a random value
    - **Code** can only be randomized by comiling the program as Position Independent Code/Position Independent Executable
       - default for shared libraries, but otherwise executable code is usually placed at fixed addresses

### ASLR values
- Linux allows 3 options:
    - 0: disabled
    - 1: randomize stack, vdso, libraries (except data? not sure...)
    - 2: enabled (including data segment - default value)
    
### checking ASLR
- compile and run demos/aslr.cpp program to check ASLR
- run gdb-peda 

    ```bash
    aslr # run aslr command
    ASLR is OFF # output
    ```

In [26]:
! echo kali | sudo -S cat /proc/sys/kernel/randomize_va_space

2


[sudo] password for kali: 

In [28]:
# let's copy and compile aslr.cpp demo program
! cp demos/aslr.cpp .

In [29]:
! cat aslr.cpp

// aslr.cpp - program to check address space layout randomization
// compile and run the program multiple times
// if you see same value for $esp, ASLR is disabled
// $ g++ -g -m 32 -o aslr.exe aslr.cpp
#include <stdio.h>
#include <iostream>
using namespace std;

int global_var = 100; // data segment
char global_name[20]; // bss segment

int main(void) {
    char buffer[] = "Hello World!";
    register int esp asm("esp");
    register int ebp asm("ebp");
    cout << "Registers info:\n";
    cout << "===============\n";
    printf("$esp = %p\n", esp);
    printf("$ebp = %p\n", ebp);
    cout << "\nData and BSS segement\n";
    cout << "======================\n";
    printf("global_var is at: %p\n", &global_var);
    printf("global_name is at: %p\n", global_name);
    cout << "\nStack variable:\n";
    cout << "===============\n";
    printf("buffer is at: %p\n", buffer);
    cout << "\nCode segment:\n";
    cout << "=============\n";
    printf("main is at %p\n", main);
    cout << "\nS

In [30]:
! g++ -m32 aslr.cpp -o aslr.exe

In [34]:
! ./aslr.exe | tee aslr2-1.txt

Registers info:
$esp = 0xffe40950
$ebp = 0xffe40968

Data and BSS segement
global_var is at: 0x5659c024
global_name is at: 0x5659c02c

Stack variable:
buffer is at: 0xffe40953

Code segment:
main is at 0x565991b9

Shared library code
printf is at 0xf7bcdf20
cout is at 0xf7f2bc00


In [35]:
! ./aslr.exe | tee aslr2-2.txt

Registers info:
$esp = 0xffdf9490
$ebp = 0xffdf94a8

Data and BSS segement
global_var is at: 0x5659f024
global_name is at: 0x5659f02c

Stack variable:
buffer is at: 0xffdf9493

Code segment:
main is at 0x5659c1b9

Shared library code
printf is at 0xf7c34f20
cout is at 0xf7f92c00


In [37]:
# run the diff of aslr2-1.txt and aslr2-2.txt
! diff aslr2-1.txt aslr2-2.txt

3,4c3,4
< $esp = 0xffe40950
< $ebp = 0xffe40968
---
> $esp = 0xffdf9490
> $ebp = 0xffdf94a8
8,9c8,9
< global_var is at: 0x5659c024
< global_name is at: 0x5659c02c
---
> global_var is at: 0x5659f024
> global_name is at: 0x5659f02c
13c13
< buffer is at: 0xffe40953
---
> buffer is at: 0xffdf9493
17c17
< main is at 0x565991b9
---
> main is at 0x5659c1b9
21,22c21,22
< printf is at 0xf7bcdf20
< cout is at 0xf7f2bc00
---
> printf is at 0xf7c34f20
> cout is at 0xf7f92c00


### Set ASLR to option 1
- randomize stack, vdso, libraries (except data?)
- actually, global data and bss segments are also randomized!

In [ ]:
# run it on terminal as you can't write in sudo interactive in Jupyter Notebook
! echo 1 | sudo tee /proc/sys/kernel/randomize_va_space

In [42]:
! echo kali | sudo -S cat /proc/sys/kernel/randomize_va_space

[sudo] password for kali: 1


In [43]:
! ./aslr.exe | tee aslr1-1.txt

Registers info:
$esp = 0xffa92ec0
$ebp = 0xffa92ed8

Data and BSS segement
global_var is at: 0x565fc024
global_name is at: 0x565fc02c

Stack variable:
buffer is at: 0xffa92ec3

Code segment:
main is at 0x565f91b9

Shared library code
printf is at 0xf7be6f20
cout is at 0xf7f44c00


In [44]:
! ./aslr.exe | tee aslr1-2.txt

Registers info:
$esp = 0xffec4850
$ebp = 0xffec4868

Data and BSS segement
global_var is at: 0x5659b024
global_name is at: 0x5659b02c

Stack variable:
buffer is at: 0xffec4853

Code segment:
main is at 0x565981b9

Shared library code
printf is at 0xf7c13f20
cout is at 0xf7f71c00


In [45]:
! diff aslr1-1.txt aslr1-2.txt

3,4c3,4
< $esp = 0xffa92ec0
< $ebp = 0xffa92ed8
---
> $esp = 0xffec4850
> $ebp = 0xffec4868
8,9c8,9
< global_var is at: 0x565fc024
< global_name is at: 0x565fc02c
---
> global_var is at: 0x5659b024
> global_name is at: 0x5659b02c
13c13
< buffer is at: 0xffa92ec3
---
> buffer is at: 0xffec4853
17c17
< main is at 0x565f91b9
---
> main is at 0x565981b9
21,22c21,22
< printf is at 0xf7be6f20
< cout is at 0xf7f44c00
---
> printf is at 0xf7c13f20
> cout is at 0xf7f71c00


### Set ASLR to option 0
- disable ASLR

In [ ]:
# run it on terminal as you can't write in sudo interactive in Jupyter Notebook
! echo 0 | sudo tee /proc/sys/kernel/randomize_va_space

In [46]:
! echo kali | sudo -S cat /proc/sys/kernel/randomize_va_space

[sudo] password for kali: 0


In [47]:
! ./aslr.exe | tee aslr0-1.txt

Registers info:
$esp = 0xffffc370
$ebp = 0xffffc388

Data and BSS segement
global_var is at: 0x56559024
global_name is at: 0x5655902c

Stack variable:
buffer is at: 0xffffc373

Code segment:
main is at 0x565561b9

Shared library code
printf is at 0xf7c54f20
cout is at 0xf7fb2c00


In [48]:
! ./aslr.exe | tee aslr0-2.txt

Registers info:
$esp = 0xffffc370
$ebp = 0xffffc388

Data and BSS segement
global_var is at: 0x56559024
global_name is at: 0x5655902c

Stack variable:
buffer is at: 0xffffc373

Code segment:
main is at 0x565561b9

Shared library code
printf is at 0xf7c54f20
cout is at 0xf7fb2c00


In [49]:
! diff aslr0-1.txt aslr0-2.txt
# no difference!

#### Allow ptrace processes
**ptrace** - allows parent process to observe and control the execution of another process
- used primarily to implement breakpoint debugging and system call tracing

In [7]:
# check the ptrace value
! echo kali | sudo -S sysctl kernel.yama.ptrace_scope

kernel.yama.ptrace_scope = 0


[sudo] password for kali: 

In [8]:
# set the ptrace value
! echo kali | sudo -S sysctl -w kernel.yama.ptrace_scope=0

kernel.yama.ptrace_scope = 0


[sudo] password for kali: 

### 2. PIE
- Position Independent Executable
- randomizes Code segment base address
- randomizes GOT (Global Offset Table for global data-variables) and PLT (Procedure Linkage Table)
    - PLT addresses are fixed during  linking to load the libc.so library codes/functions
    - shared libraries are position independent code (PIC); they don't know where they are being loaded
- disable PIE in gcc using **-no-pie** flag

### 3. Executable Stack Protection (NX) also (DEP)
- DEP - Data Execution Prevention
- in newer gcc, programs must be compiled by disabling stack protector to execute code injected into stack
- check executable stack:
`$ readelf -l <filename>`
- note the line: 
GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RW  0x10
RW 
- RW - Read and Write but not Executable
- RWE - Read, Write, and Executable



In [9]:
%%bash
in=./demo-programs/hello.cpp
out=hello
g++ -g -m32 -o $out $in 

readelf -l ./demo-programs/aslr | grep GNU_STACK

  GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RW  0x10


#### enable RWX/disable DEP
- compile the program using -z execstack switch of gcc

In [10]:
%%bash
in=./demo-programs/hello.cpp
out=hello
g++ -g -m32 -z execstack -o $out $in 

readelf -l $out | grep GNU_STACK

  GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RWE 0x10


### 4. Stack Canaries
- "canary in a coal mine" to detect carbon monoxide by using a warm blooded animal like a bird
- place a small random integer just before the stack return pointer
    - if the value is overwritten/corrupted during the function call then there's a buffer overflow attempt
- use **-fno-stack-protector** flag in gcc to disable canaries